In [1]:
%matplotlib inline
import numpy as np
import folium
from folium import plugins
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

In [138]:
df = pd.read_csv("../../data/peseudo_rent_data.csv")

In [3]:
np.random.seed(seed=2019)

In [139]:
df["month"] = np.random.randint(1, 13, len(df))

In [5]:
df.head(2)

,No,都道府県,市区町村,町丁目,築年,築月,構造,マンション最上階数,間取別平均平米単価①,間取別平均平米単価②,...,間取別平均平米単価⑤,物件緯度,物件経度,最寄駅,最寄駅緯度,最寄駅経度,始点ノード,終点ノード,最寄駅までの距離,month
0,1,東京都,世田谷区,喜多見７丁目,1988,3,ＲＣ,2,2812.2742,2311.9321,...,2634.3530,35.631294,139.594005,喜多見,35.636745,139.587513,1.395870e+14,1.395930e+14,1096.93,9
1,2,東京都,世田谷区,喜多見７丁目,2005,3,鉄骨造,3,3006.8074,2633.6113,...,2733.7278,35.632051,139.592723,喜多見,35.636745,139.587513,1.395870e+14,1.395920e+14,930.57,3


In [140]:
df["rent_price_index"] = df["間取別平均平米単価①"] / max(df["間取別平均平米単価①"])

In [141]:
#df = df[['最寄駅', '最寄駅緯度', '最寄駅経度','month', 'rent_price_index']]
df = df[['物件緯度', '物件経度','month', 'rent_price_index']]

In [142]:
#station_df = df.groupby(['最寄駅', '最寄駅緯度', '最寄駅経度','month']).mean()
station_df = df

In [143]:
station_df= station_df.reset_index(drop=False)

In [144]:
station_df = station_df.sort_values('month')

In [146]:
station_df.head(2)

,index,物件緯度,物件経度,month,rent_price_index
3565,3565,35.671705,139.603124,1,0.491533
816,816,35.625649,139.638543,1,0.546328


In [147]:
#heat_data = [[[row['最寄駅緯度'],row['最寄駅経度'], row["rent_price_index"]] for index, row in station_df[station_df.month == month].iterrows()] for month in range(1, 13)]
heat_data = [[[row['物件緯度'],row['物件経度'], row["rent_price_index"]] for index, row in station_df[station_df.month == month].iterrows()] for month in range(1, 13)]

#heat_data = [[row['物件緯度'],row['物件経度'], ] for index, row in _df.iterrows()]
#geometry = [Point(lonlat) for lonlat in zip(df["物件経度"], df["物件緯度"])]
#crs = {'init': 'epsg:4326'} 
#heat_data = [[[row['lat'],row['lon'], row["平成"+year+"年"]] for index, row in prefecture_df.iterrows()] for year in time_series]

In [148]:
dt = datetime(2015, 1, 1, 0, 0, 0, 0)
time_index = [
    (dt + t * relativedelta(months=1)).strftime('%Y-%m-%d') for t in range(len(heat_data))
]


In [149]:
time_index

['2015-01-01',
 '2015-02-01',
 '2015-03-01',
 '2015-04-01',
 '2015-05-01',
 '2015-06-01',
 '2015-07-01',
 '2015-08-01',
 '2015-09-01',
 '2015-10-01',
 '2015-11-01',
 '2015-12-01']

In [150]:
zoom_start = 15
m = folium.Map(location=[35.643554, 139.671146],
                   tiles='https://cyberjapandata.gsi.go.jp/xyz/seamlessphoto/{z}/{x}/{y}.jpg',
                   attr='国土地理院 写真',
                   zoom_start=zoom_start)

In [151]:
hm = plugins.HeatMapWithTime(data=heat_data, index=time_index, auto_play=True, max_opacity=0.3, radius=100)
hm.add_to(m)

In [152]:
m